# Calculating SNR and CNR with Gaussian Mixture Models
## Phantom Validation

The signal-to-noise ratio (SNR) and contrast-to-noise ratio (CNR) of a 3D stack of greyscale images can be determined from the histogram of the grey values in the stack. Each material in the image (e.g. air, wax, tissue) is represented by a Gaussian distribution of grey values with a mean $\mu$, standard deviation $\sigma$ and weight <i><b>w</i></b>. SNR and CNR then calculated by the following equations:

\begin{equation*}
SNR = \frac{\mu_A}{\sigma_B}
\end{equation*}

\begin{equation*}
CNR = \frac{|\mu_A - \mu_B|}{\sqrt{\sigma_A^2+\sigma_B^2}}
\end{equation*}

A phantom was created to validate this SNR and CNR measure. An example 3D micro-CT image stack kindly provided by Harry Rossides was segmented to produce three masks, each corresponding to air, wax and tissue respectively. Each pixel within each mask was assigned a random grey value from a normal distribution with a known $\mu$ and $\sigma$. Several tests are then performed with this phantom with known Gaussian distributions of each material.

In [2]:
%run "Phantom_Validation_Hidden.py" # Imports functions and libraries to calculate Q

# Phantom Generation

The first slice of each mask can be viewed in the cell below.

In [ ]:
image_names = ['Air','Wax','Tissue']
fig = plt.figure(figsize = [15.,5.])
plt.style.use('grayscale')

for i in range(0,len(image_names)):
    plt.subplot(1,3,i+1)
    plt.title(image_names[i]+' masked image',fontsize=16)
    plt.imshow(io.imread(os.path.join(test_dir, "{}.tif".format(image_names[i])))[0]) # first slice only
    plt.axis('off')

plt.show()


## Assigning new grey values

The grey values of the pixels in each mask were replaced with random values from Gaussian distributions of known $\mu$ and $\sigma$. 

In [ ]:
mu_phantom = [8, 35, 82] # phantom mu for air, wax, tissue
sigma_phantom = [7, 6, 16] # phantom sigma for air, wax, tissue

I_phantom = create_phantom.create_phantom(mu_phantom, sigma_phantom, test_dir)
create_phantom.show_phantom(I_phantom)

# Fitting Gaussian distributions

SNR and CNR were calculated by fitting three Gaussian distributions to the phantom image stack histogram (one for each material). Firstly, the accuracy of the fitted Gaussian distributions was assessed by comparing the properties of the fitted and phantom distributions.

Percentage error was calculated with the following equations:

\begin{equation*}
\% \;error \;in \; \mu= 100 \cdot \frac{\mu_{phantom} - \mu}{\mu_{phantom}}
\end{equation*}

\begin{equation*}
\% \;error \;in \; \sigma = 100 \cdot \frac{\sigma_{phantom} - \sigma}{\sigma_{phantom}}
\end{equation*}

In [ ]:
mu_fitted, sigma_fitted = QM_fit.QM_fit(I_phantom, 3, img_fname = "NA", save_results = False) # fit Gaussian distributions

pct_diff_mu = map(calc_pct_diff, mu_phantom, mu_fitted) # calculate % diff in mu
pct_diff_sigma = map(calc_pct_diff, sigma_phantom, sigma_fitted) # calculate % diff in sigma

# Display results
df = pd.DataFrame([pct_diff_mu, pct_diff_sigma], 
                  index = ["% difference $\mu$","% difference $\sigma$"], 
                  columns = ["Air", "Wax", "Tissue"]) # convert to pandas df for pretty displays
df_style = df.style.set_caption("Percentage Difference between Fitted and Phantom Distributions")
display(df_style)


# Calculating image quality

The Gaussian distributions were used to calculate SNR and CNR between Gaussians A and B, where A and B were all the different combinations of fitted Gaussians. Here we have three materials represented in the image, so 3 Gaussians were fitted and SNR was calculated between all of them.

$$ SNR = \frac{\mu_{A}}{\sigma_{B}}  $$

$$ CNR = \frac{\mu_{A} - \mu_{B}}{\sqrt{\sigma_{A}^2 + \sigma_{B}^2}} $$

In this case, Gaussian 0 was air, Gaussian 1 was wax and Gaussian 2 was tissue.

In [ ]:
SNR_CNR_df = QM_calc.QM_calc(mu_fitted, sigma_fitted, out_dir = "NA", save_results = False)

# Manipulating phantom distributions

The grey value distributions of the phantom image affect the accuracy of the fitted Gaussian distributions and the image quality. Below is an interactive model where the means and standard deviations of the three distributions in the phantoms can be set. Run the following cell and adjust the values of $\mu$ and $\sigma$.


In [ ]:
mu_0 = widgets.IntSlider(description = '$\mu_0$', min = 0, max = 255)
mu_1 = widgets.IntSlider(description = '$\mu_1$', min = 0, max = 255)
mu_2 = widgets.IntSlider(description = '$\mu_2$', min = 0, max = 255)

sigma_0 = widgets.IntSlider(description = '$\sigma_0$', min = 5, max = 50)
sigma_1 = widgets.IntSlider(description = '$\sigma_1$', min = 5, max = 50)
sigma_2 = widgets.IntSlider(description = '$\sigma_2$', min = 5, max = 50)

mu_box = widgets.VBox([mu_0, mu_1, mu_2])
sigma_box = widgets.VBox([sigma_0, sigma_1, sigma_2])  
widgets.HBox([mu_box, sigma_box])

## Determining fit accuracy and image quality

Run the following cell to calculate the fit accuracy and image quality based on the grey value distributions chosen before.

\begin{equation*}
\% \;error \;in \; \mu= 100 \cdot \frac{\mu_{interact} - \mu}{\mu_{interact}}
\end{equation*}

\begin{equation*}
\% \;error \;in \; \sigma = 100 \cdot \frac{\sigma_{interact} - \sigma}{\sigma_{interact}}
\end{equation*}

In [ ]:
mu_interact = [mu_0.value, mu_1.value, mu_2.value] # widget outputs
sigma_interact = [sigma_0.value, sigma_1.value, sigma_2.value] # widget outputs

# Create phantom
I_phantom = create_phantom.create_phantom(mu_interact, sigma_interact, test_dir)
create_phantom.show_phantom(I_phantom)

# Fit GMM, calculate and display results
mu_fitted, sigma_fitted = QM_fit.QM_fit(I_phantom, 3, img_fname = "NA", save_results = False) # fit Gaussian distributions
pct_diff_mu = map(calc_pct_diff, mu_interact, mu_fitted) # calculate % diff in mu
pct_diff_sigma = map(calc_pct_diff, sigma_interact, sigma_fitted) # calculate % diff in sigma

SNR_CNR_df = QM_calc.QM_calc(mu_fitted, sigma_fitted, out_dir = "NA", save_results = False) # calc SNR and CNR

# Display results
df = pd.DataFrame([pct_diff_mu, pct_diff_sigma], 
                  index = ["% difference $\mu$","% difference $\sigma$"], 
                  columns = ["Air", "Wax", "Tissue"]) # convert to pandas df for pretty displays
df_style = df.style.set_caption("Percentage Difference between Fitted and Phantom Distributions")
display(df_style)

## Effect of contrast on fit accuracy and image quality STOPPED HERE 04-12-2019

The contrast between tissue and wax is the difference between means of the grey value distributions for tissue and wax. This can be altered by changing the mean of the grey value distribution for tissue. The following cell calculates the percentage difference in fitted and assigned $\mu$ and $\sigma$ for a range of $\mu_{tissue}$ between 40 and 100. The grey value distributions of air and wax were kept constant at the following values:

$\mu_{air}$ = 8, $\sigma_{air}$ = 7

$\mu_{wax}$ = 35, $\sigma_{wax}$ = 6

$\mu_{tissue}$ = variable, $\sigma_{tissue}$ = 17

In [ ]:
# Define grey value distributions
mu_tissue_contrast = np.linspace(60, 100, 5) # 10 values of mu for tissue between 40 and 100
sigma_phantom = [7, 6, 16] # phantom sigma for air, tissue, wax
pct_diff_mu_results = []
pct_diff_sigma_results = []
CNR = []

for mu in mu_tissue_contrast:
    # Create phantom
    print("\nCreating phantom with mu of tissue =", mu)
    mu_phantom = [8, 35, mu]
    I_phantom = create_phantom.create_phantom(mu_phantom, sigma_phantom, test_dir)
    
    # Fit GMM, calculate results
    mu_fitted, sigma_fitted = QM_fit.QM_fit(I_phantom, 3, img_fname = "NA", save_results = False, show_plots = False)
    pct_diff_mu = list(map(calc_pct_diff, mu_phantom, mu_fitted)) # calculate % diff in mu
    pct_diff_sigma = list(map(calc_pct_diff, sigma_phantom, sigma_fitted)) # calculate % diff in sigma
    pct_diff_mu_results.append(pct_diff_mu)
    pct_diff_sigma_results.append(pct_diff_sigma)
    CNR.append((mu - 35)/np.sqrt(6**2 + 16**2))
    


Creating phantom with mu of tissue = 60.0
Phantom means are [8, 35, 60.0]
Phantom standard deviations are [7, 6, 16]
Phantom successfully created


The following cell plots the percentage differences in fitted and assigned $\mu$ and $\sigma$ for different values of $\mu_{tissue}$. 

<b><u>Accuracy of fitted grey value distributions was found to increase as contrast increases (separation between tissue and wax distributions). The relationship between image quality Q and contrast was approximately linear, where Q and contrast were both determined between wax and tissue grey value distributions.</b></u>

In [1]:
contrast_plot(mu_tissue_contrast, pct_diff_mu_results, pct_diff_sigma_results, CNR)

NameError: name 'contrast_plot' is not defined

## Effect of distribution spread on fit accuracy and image quality

Distributions with large spreads (standard deviations) close to each other are more likely to overlap. Large areas of overlap between distributions reduce the accuracy of Gaussian distribution fitting and reduce image quality. The standard deviation of the grey value distributions were scaled by a factor of 0.1 to 2 and the means were kept constant. The assigned distributions are shown below:

$\sigma$ scaling factor is a value between 0.1 and 2.

$\mu_{air}$ = 8, $\sigma_{air}$ = 7 x $\sigma$ scaling factor

$\mu_{wax}$ = 35, $\sigma_{wax}$ = 6 x $\sigma$ scaling factor

$\mu_{tissue}$ = 82, $\sigma_{tissue}$ = 17 x $\sigma$ scaling factor


In [ ]:
sigma_factor = np.linspace(0.1,2,20) # range of mu for tissue
mu_phantom = [8, 35, 82] # phantom mu for air, tissue, wax
pct_diff_mu = np.zeros((len(sigma_factor),3))
pct_diff_sigma = np.zeros((len(sigma_factor),3))
Q_spread = np.zeros((len(sigma_factor)))

for i,sigma in enumerate(sigma_factor):
    sigma_phantom = [7*sigma, 6*sigma, 17*sigma] # phantom mu for air, tissue, wax
    diff_mu, diff_sigma, Q = create_phantom_shortcut(mu_phantom, sigma_phantom, output = False, return_vals = True)    
    pct_diff_mu[i] = diff_mu
    pct_diff_sigma[i] = diff_sigma
    Q_spread[i] = Q

The following cell plots the percentage differences in fitted and assigned $\mu$ and $\sigma$ for different values of $\sigma$ scaling factor.

<b><u>Accuracy of fitted grey value distributions was found to decrease as standard deviations of grey value distributions for each material increased. The relationship between image quality Q and standard deviation was found to be inversely variable, i.e. $Q = f(\frac{k}{\sigma scaling factor})$. Here, Q was calculated between tissue and wax grey value distributions.</b></u>

In [ ]:
plt.figure(figsize = (15,5))
sns.set_style('white')

plt.subplot(1,3,1)
plot_pct_diff(sigma_factor, pct_diff_mu, "% Difference in $\mu$", "$\sigma$ scaling factor")
plt.subplot(1,3,2)
plot_pct_diff(sigma_factor, pct_diff_sigma, "% Difference in $\sigma$", "$\sigma$ scaling factor")
plt.subplot(1,3,3)
plot_pct_diff(sigma_factor, Q_spread, "Image quality Q", "$\sigma$ scaling factor", legend = False)

In [ ]:
pct_diff_mu


# Dependencies

Dependencies to recreate this environment.

In [ ]:
%reload_ext watermark
%watermark -v -m -p matplotlib,skimage,skimage.transform,numpy,sklearn.mixture,scipy.stats,pandas